In [3]:
import numpy as np
import torch 
torch.set_printoptions(edgeitems =2, linewidth =75)
import torch.optim as optim
t_c =[0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u =[35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)
t_un = 0.1*t_u


def model(t_u, w, b):
    return w*t_u+b 

def loss_fn(t_p, t_c):
    square_diffs = (t_p -t_c)**2
    return square_diffs.mean()

def dloss_fn(t_p, t_c):
    dsq_diffs = 2*(t_p - t_c) / t_p.size(0)
    return dsq_diffs

def dmodel_dw(t_u, w, b):
    return t_u

def dmodel_db(t_u, w, b):
    return 1.0
def grad_fn(t_u, t_c, t_p,w, b):
    dloss_dtp = dloss_fn(t_p, t_c)
    dloss_dw = dloss_dtp * dmodel_dw(t_u, w, b)
    dloss_db = dloss_dtp * dmodel_db(t_u, w, b)
    return torch.stack([dloss_dw.sum(), dloss_db.sum()])


In [8]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,train_t_c, val_t_c):
    for epoch in range(1, n_epochs+1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)

        with torch.no_grad(): # context manager
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad == False

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <=3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f}, Validation loss {val_loss.item():.4f}")
    return params

In [5]:
n_samples = t_u.shape[0]
n_val = int(0.2*n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([3, 1, 2, 8, 0, 7, 6, 9, 5]), tensor([ 4, 10]))

In [6]:
train_t_u = t_u[train_indices]
train_t_c = t_c [train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c [val_indices]

train_t_un = 0.1*train_t_u
val_t_un = 0.1*val_t_u

In [9]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr = learning_rate)

training_loop(3000, optimizer, params, train_t_un,val_t_un, train_t_c, val_t_c )

Epoch 1, Training loss 72.7406, Validation loss 114.6712
Epoch 2, Training loss 40.2682, Validation loss 46.4569
Epoch 3, Training loss 33.9582, Validation loss 27.9884
Epoch 500, Training loss 7.0645, Validation loss 6.8510
Epoch 1000, Training loss 3.4611, Validation loss 4.2857
Epoch 1500, Training loss 2.9574, Validation loss 3.5832
Epoch 2000, Training loss 2.8870, Validation loss 3.3564
Epoch 2500, Training loss 2.8772, Validation loss 3.2767
Epoch 3000, Training loss 2.8758, Validation loss 3.2475


tensor([  5.2908, -16.8837], requires_grad=True)

In [ ]:
def calc_forward(t_u, t_c, is_train:bool):
    with torch.set_grad_enabled(is_train):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
    return loss